<a href="https://colab.research.google.com/github/arun-arunisto/Tensorflow_Tutorial/blob/todo/Genrating_Texts_With_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Recurrent Neural Networks
- very useful when it comes to the processing of sequential data like text.

In this, guide we are going to use LSTM (Long-Short-Term-Memory) for generating texts like shakespare.

- We train a neural network to write texts similar to those famous poet.
- recurrent neural networks and LSTMs in particular have a short term memory we can train it to "guess" the next letter based on the letters thats came before.

## Loading Text

In [1]:
#importing the modules
import random
import numpy as np
import tensorflow as tf

## Loading dataset

we are going to use data from tensorflow

In [2]:
filepath = tf.keras.utils.get_file("shakespare.txt",
                                   'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8')

1115394/1115394 [==============================] - 0s 0us/step


## Preparing Data

The problem that we have right now is that we have text data. we are not able to train the neural network with letters or sentences. so, first we need to convert all these texts into numerical values. and then convert the resulting numbers back into text.

In [3]:
#converting it into number
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [4]:
text

"first citizen:\nbefore we proceed any further, hear me speak.\n\nall:\nspeak, speak.\n\nfirst citizen:\nyou are all resolved rather to die than to famish?\n\nall:\nresolved. resolved.\n\nfirst citizen:\nfirst, you know caius marcius is chief enemy to the people.\n\nall:\nwe know't, we know't.\n\nfirst citizen:\nlet us kill him, and we'll have corn at our own price.\nis't a verdict?\n\nall:\nno more talking on't; let it be done: away, away!\n\nsecond citizen:\none word, good citizens.\n\nfirst citizen:\nwe are accounted poor citizens, the patricians good.\nwhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them let us revenge this with\nour pikes, ere we become rakes: for the gods know i\nspeak this in hunger 

Next we are going to create a sorted set of all the unique characters that occur in the text. in set values apperas only once no duplicates are allowed, so this is a good way to filter out the characters.

In [5]:
characters = sorted(set(text))

Next we defining two structures for converting the values, both will be dictionaries that enumerate the characters.

In [7]:
#creating a dictionary the character and indices are the values
char_to_index = dict((c, i) for i, c in enumerate(characters))

In [8]:
#creating a dictionary with indices and character are the values
index_to_char = dict((i, c) for i, c in enumerate(characters))

Next we define how long a sequence shall be and how many characters we will step further to start the next sentence.

In [9]:
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_char = []

above we try to do take sentences and to take next character for training data. so, next we are going to iterate through elements going to add data into sentences and next_char list for training data

In [10]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
  sentences.append(text[i: i + SEQ_LENGTH])
  next_char.append(text[i+SEQ_LENGTH])

In [14]:
sentences

['first citizen:\nbefore we proceed any fur',
 'st citizen:\nbefore we proceed any furthe',
 'citizen:\nbefore we proceed any further, ',
 'izen:\nbefore we proceed any further, hea',
 'n:\nbefore we proceed any further, hear m',
 'before we proceed any further, hear me s',
 'ore we proceed any further, hear me spea',
 ' we proceed any further, hear me speak.\n',
 ' proceed any further, hear me speak.\n\nal',
 'oceed any further, hear me speak.\n\nall:\n',
 'ed any further, hear me speak.\n\nall:\nspe',
 'any further, hear me speak.\n\nall:\nspeak,',
 ' further, hear me speak.\n\nall:\nspeak, sp',
 'rther, hear me speak.\n\nall:\nspeak, speak',
 'er, hear me speak.\n\nall:\nspeak, speak.\n\n',
 ' hear me speak.\n\nall:\nspeak, speak.\n\nfir',
 'ar me speak.\n\nall:\nspeak, speak.\n\nfirst ',
 'me speak.\n\nall:\nspeak, speak.\n\nfirst cit',
 'speak.\n\nall:\nspeak, speak.\n\nfirst citize',
 'ak.\n\nall:\nspeak, speak.\n\nfirst citizen:\n',
 '\n\nall:\nspeak, speak.\n\nfirst citizen:\ny

In [15]:
next_char

['t',
 'r',
 'h',
 'r',
 'e',
 'p',
 'k',
 '\n',
 'l',
 's',
 'a',
 ' ',
 'e',
 '.',
 'f',
 's',
 'c',
 'i',
 'n',
 'y',
 ' ',
 'e',
 'l',
 'r',
 'o',
 'e',
 'r',
 'h',
 ' ',
 ' ',
 'e',
 'h',
 ' ',
 ' ',
 'm',
 'h',
 '\n',
 'l',
 'r',
 'o',
 'e',
 ' ',
 's',
 'v',
 '.',
 'f',
 's',
 'c',
 'i',
 'n',
 'f',
 's',
 ' ',
 'u',
 'n',
 ' ',
 'i',
 ' ',
 'r',
 'u',
 'i',
 'c',
 'e',
 'e',
 'm',
 't',
 't',
 ' ',
 'o',
 'e',
 '\n',
 'l',
 'w',
 'k',
 'w',
 ',',
 'e',
 'n',
 "'",
 '\n',
 'i',
 't',
 'i',
 'z',
 ':',
 'e',
 'u',
 'k',
 'l',
 'i',
 ' ',
 'd',
 'e',
 'l',
 'a',
 ' ',
 'r',
 'a',
 'o',
 ' ',
 'n',
 'r',
 'e',
 'i',
 't',
 ' ',
 'r',
 'c',
 '\n',
 'l',
 '\n',
 ' ',
 'r',
 't',
 'k',
 'g',
 'n',
 ';',
 'e',
 'i',
 'b',
 'd',
 'e',
 'a',
 'y',
 'a',
 'y',
 '\n',
 'c',
 'd',
 'i',
 'z',
 ':',
 'n',
 'w',
 'd',
 'g',
 'd',
 'i',
 'z',
 's',
 '\n',
 'r',
 ' ',
 't',
 'e',
 '\n',
 ' ',
 'e',
 'c',
 'u',
 'e',
 'p',
 'r',
 'i',
 'z',
 's',
 't',
 ' ',
 't',
 'c',
 'n',
 'g',
 'd',
 'w',
 

we are going to create two numpy arrays full of zeros. the data type of those is bool, which stands for boolean whereever characters apperas in a cretain sentence ata a certain position we will set it to a one or a True.

In [16]:
#one dimension for the sentences
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool)

#one dimension for the positions of the chracters
y = np.zeros((len(sentences), len(characters)), dtype=np.bool)

<ipython-input-16-b5cff870c826>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool)
<ipython-input-16-b5cff870c826>:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(characters)), dtype=np.bool)


In [17]:
#within the sentence and one dimension to specify which character is at this position
for i, satz in enumerate(sentences):
  for t, char in enumerate(satz):
    x[i, t, char_to_index[char]] = 1
  y[i, char_to_index[next_char[i]]] = 1

In [18]:
x

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False,  True, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [19]:
y

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

So, finally our training data is prepared, next we need to build our neural network

In [20]:
#importing modules
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

In [21]:
#defining model
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

structure is simple. the inputs immediately flow into our LSTM layer with 128 neurons. the input shape is the length of a sentence times the amount of characters. the character which shall follow will be set to True or one. this layer is followed by a Dense hidden layer, which just increases complexity. In the end we use the Softmax activation function in order to make our results add up to one. this will give us the probability for each character

In [22]:
#compiling the model and train it with our training data that we prepared above
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

#we are choosing batch size of 256 and four eepochs. this means that our model is going to see the same data four times
model.fit(x, y, batch_size=256, epochs=4)

Epoch 1/4
1453/1453 [==============================] - 258s 175ms/step - loss: 2.4852
Epoch 2/4
1453/1453 [==============================] - 243s 167ms/step - loss: 2.1403
Epoch 3/4
1453/1453 [==============================] - 241s 166ms/step - loss: 2.0160
Epoch 4/4
1453/1453 [==============================] - 244s 168ms/step - loss: 1.9292


Now our model is now trained but it only outputs the probabilities for the next character, so, need some additional functions to make our script generate some reasonable text.

In [23]:
def sample(preds, temperature=1.0):
  preds = np.array(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds/np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

It basically just picks one of the characters from the output. As parameters it takes the result of the prediction and a temperature. This temperature indicates how risky the pick shall be. If we have a high temperature, we will pick one of the less likely characters. A low temperature will cause a conservative choice.

In [24]:
#next we are going to write our final function this function generates the final text
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

Again, it is less complicated than it looks. We basically choose a random starting position within the text because we need some starting text in order to predict the “next” character. So basically the first SEQ_LENGTH amount of characters will be copied from the original text. But we could just cut them off afterwards and we would end up with text that is completely generated by our neural network.

So we choose some random starting text and then we run a for loop in the range of the length that we want. We can generate a text with 100 characters or one with 20,000. We then convert our sentence into the desired input format that we already talked about. The sentence is now an array with ones or Trues, wherever a character occurs. Then we use the predict method of our model, to predict the likelihoods of the next characters. Then we make use of our sample helper function. In this function we also have a temperature parameter, which we can pass to that helper function. Of course the result we get needs to be converted from the numerical format into a readable character. Once this is done, we add the character to our generated text and repeat the process, until we reach the desired length.

In [26]:
print(generate_text(300, 0.2))

heir issue not
being gracious, than they sore the comes
and the sing to the conest the raget to thee sore.

comens:
i ware the come to the sient the comes to thee the with
and the sore the coust the to the counter there
then the sould the master to the sore sores
and the to the sore the songer thee the camester
with the come so prome the 
